In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.cuda.amp import autocast, GradScaler
from collections import Counter
import string

#Key Changes For Faster Training:
#Batching and Padding: prepare_batch prepares input and target batches with padding and corresponding sequence lengths.
#Packed Sequences: The forward method handles packed sequences to avoid unnecessary computation on padded tokens.
#Gradient Clipping: Applied gradient clipping to avoid exploding gradients.
#Mixed Precision Training: Using torch.cuda.amp with automatic casting and gradient scaling to accelerate training on GPUs with mixed precision.
#Reduced Hidden Size: Reduced the hidden size to 64 for faster training.
#Ignored Padding in Loss: ignore_index=vocab['<PAD>'] ensures padding tokens are ignored during loss calculation

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Chatbot Model Definition
class ChatbotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(ChatbotModel, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        # Embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size)

        # LSTM layer
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, hidden_state, lengths):
        embedded = self.embedding(input_seq)
        packed_embedded = pack_padded_sequence(embedded, lengths, batch_first=True, enforce_sorted=False)

        packed_output, hidden_state = self.lstm(packed_embedded, hidden_state)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        output = self.fc(output)
        return output, hidden_state

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size).to(device),
                torch.zeros(self.n_layers, batch_size, self.hidden_size).to(device))


# Tokenize and preprocess text data
def tokenize(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return text.split()

# Build a vocabulary from the dataset
def build_vocab(dataset):
    all_words = []
    for sentence in dataset:
        all_words.extend(tokenize(sentence))

    vocab = {word: i for i, word in enumerate(set(all_words))}
    vocab['<PAD>'] = len(vocab)  # Padding token
    vocab['<SOS>'] = len(vocab)  # Start of sentence
    vocab['<EOS>'] = len(vocab)  # End of sentence
    return vocab

# Prepare batch with padding
def prepare_batch(input_tensors, target_tensors, batch_size):
    idx = np.random.choice(len(input_tensors), batch_size, replace=False)
    batch_inputs = [input_tensors[i] for i in idx]
    batch_targets = [target_tensors[i] for i in idx]

    # Pad the input and target sequences to the same length
    padded_inputs = pad_sequence(batch_inputs, batch_first=True, padding_value=vocab['<PAD>'])
    padded_targets = pad_sequence(batch_targets, batch_first=True, padding_value=vocab['<PAD>'])
    lengths = [len(tensor) for tensor in batch_inputs]

    return padded_inputs, padded_targets, lengths

# Training function
def train(chatbot_model, input_tensor, target_tensor, optimizer, criterion, batch_size, lengths, scaler):
    hidden_state = chatbot_model.init_hidden(batch_size)

    optimizer.zero_grad()

    input_tensor = input_tensor.to(device)
    target_tensor = target_tensor.to(device)

    with autocast():
        output, hidden_state = chatbot_model(input_tensor, hidden_state, lengths)

        output = output.view(-1, output.size(-1))  # (batch_size * seq_len, vocab_size)
        target_tensor = target_tensor.view(-1)     # (batch_size * seq_len)

        loss = criterion(output, target_tensor)

    scaler.scale(loss).backward()

    # Gradient clipping to avoid exploding gradients
    torch.nn.utils.clip_grad_norm_(chatbot_model.parameters(), max_norm=2.0)

    scaler.step(optimizer)
    scaler.update()

    return loss.item()

# Generate response function
def generate_response(chatbot_model, input_seq, vocab, max_len=20):
    chatbot_model.eval()

    input_tensor = torch.tensor([vocab[word] for word in tokenize(input_seq)], dtype=torch.long).unsqueeze(0).to(device)
    hidden_state = chatbot_model.init_hidden(1)  # Batch size of 1 for inference

    response = []
    for _ in range(max_len):
        output, hidden_state = chatbot_model(input_tensor, hidden_state, [len(input_tensor[0])])

        _, top_word = torch.max(output[:, -1, :], dim=1)

        word = list(vocab.keys())[list(vocab.values()).index(top_word.item())]
        response.append(word)

        if word == '<EOS>':
            break

        input_tensor = torch.tensor([[top_word.item()]], dtype=torch.long).to(device)

    return ' '.join(response[:-1])  # Remove <EOS> from the response

Using device: cuda


In [76]:
# Example dataset
with open('moby_dick.txt', 'r', encoding='utf-8') as f:
    dataset = f.readlines()
print(dataset[:10])

# Build vocab and tokenize the data
vocab = build_vocab(dataset)

# Convert dataset to tensors
input_tensors = []
target_tensors = []
for sentence in dataset:
    tokenized = tokenize(sentence)
    input_tensors.append(torch.tensor([vocab['<SOS>']] + [vocab[word] for word in tokenized], dtype=torch.long))
    target_tensors.append(torch.tensor([vocab[word] for word in tokenized] + [vocab['<EOS>']], dtype=torch.long))

['\ufeff  ETYMOLOGY.\n', '\n', '\n', '  (Supplied by a Late Consumptive Usher to a Grammar School.)\n', '\n', '  The pale Usher—threadbare in coat, heart, body, and brain; I see him\n', '  now. He was ever dusting his old lexicons and grammars, with a queer\n', '  handkerchief, mockingly embellished with all the gay flags of all the\n', '  known nations of the world. He loved to dust his old grammars; it\n', '  somehow mildly reminded him of his mortality.\n']


In [100]:
# Define hyperparameters
input_size = len(vocab)
hidden_size = 128
output_size = len(vocab)
n_layers = 2
batch_size = 32
learning_rate = 0.01
epochs = 200

# Initialize model, loss function, and optimizer
chatbot_model = ChatbotModel(input_size, hidden_size, output_size, n_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=vocab['<PAD>']).to(device)
optimizer = optim.Adam(chatbot_model.parameters(), lr=learning_rate)
scaler = GradScaler()

C:\Users\calie\AppData\Local\Temp\ipykernel_18880\2700925018.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [101]:
# Training loop
for epoch in range(epochs):
    total_loss = 0
    chatbot_model.train()

    for _ in range(len(input_tensors) // batch_size):
        batch_input, batch_target, lengths = prepare_batch(input_tensors, target_tensors, batch_size)
        loss = train(chatbot_model, batch_input, batch_target, optimizer, criterion, batch_size, lengths, scaler)
        total_loss += loss

    print(f'Epoch {epoch}, Loss: {total_loss / (len(input_tensors) // batch_size)}')

C:\Users\calie\AppData\Local\Temp\ipykernel_18880\722915003.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 0, Loss: 6.9228547668457034
Epoch 1, Loss: 6.349125294155544
Epoch 2, Loss: 6.017546099203604
Epoch 3, Loss: 5.778509854917173
Epoch 4, Loss: 5.615060405731201
Epoch 5, Loss: 5.462915462917752
Epoch 6, Loss: 5.287902078275327
Epoch 7, Loss: 5.166946064277932
Epoch 8, Loss: 5.073582969948098
Epoch 9, Loss: 4.932626134377939
Epoch 10, Loss: 4.858662592569987
Epoch 11, Loss: 4.7833669330455635
Epoch 12, Loss: 4.702167423389576
Epoch 13, Loss: 4.643877285851373
Epoch 14, Loss: 4.64052464167277
Epoch 15, Loss: 4.582711551101119
Epoch 16, Loss: 4.526378768638328
Epoch 17, Loss: 4.443266615691009
Epoch 18, Loss: 4.3906854693094886
Epoch 19, Loss: 4.338305949105157
Epoch 20, Loss: 4.216825308269924
Epoch 21, Loss: 4.219532985687255
Epoch 22, Loss: 4.257833579028095
Epoch 23, Loss: 4.239511962113557
Epoch 24, Loss: 4.153317638326574
Epoch 25, Loss: 4.059692322059914
Epoch 26, Loss: 3.998166763164379
Epoch 27, Loss: 3.943091046368634
Epoch 28, Loss: 3.8598095879731353
Epoch 29, Loss: 3.878

In [114]:
# Generate a response
user_input = ("how are you")
response = generate_response(chatbot_model, user_input, vocab)
print("Bot:", response)

Bot: get to get out of the devil many other are
